In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab
! pip install git+https://github.com/huggingface/huggingface_hub#egg=huggingface-hub["fastai"]

In [ ]:
#default_exp huggingface

In [ ]:
#all_slow

In [ ]:
#export
from huggingface_hub import *
from huggingface_hub.fastai_utils import *
from typing import Optional

In [ ]:
#hide
from nbdev.showdoc import *

# Using the 🤗 Hugging Face Hub to share and load models

> Integration with the Hugging Face Hub

## Why share to the Hugging Face Hub

The Hub is a central platform where anyone can share and explore models, datasets, and ML demos. It aims to build the most extensive collection of Open Source models, datasets, and demos. 

The "solve AI" problem won't be solved by a single company, but by a culture of sharing knowledge and resources. This is something that fastai has taken to the next level 🌈. 

You can amplify your impact 🚀. By sharing your fastai Learner to the Hub, you will expose it to over 100,000 users. 

Here are some facts about the Hugging Face Hub:

- There are over 40,000 public models.
- There are models for Natural Language Processing, Computer Vision, Audio/- - Speech, and Reinforcement Learning!
- There are models for over 180 languages.
- Any ML library can leverage the Hub: from TensorFlow and PyTorch to advanced integrations with fastai 🔥 like this one!
- You can find almost 4,500 datasets.


To get know more about the Hub you can check [this introduction](https://github.com/huggingface/education-toolkit/blob/main/01_huggingface-hub-tour.md).

You can find all the fastai models in the Hugging Face Hub by filtering the [huggingface.co/models](https://huggingface.co/models) webpage by the fastai library as in the image below.

<img src="images/hf_hub_fastai.png" alt="hf_hub_fastai" width="800" />

## Installation

Install `huggingface_hub`. Additionally, we require the following packages for our functions to work:
- toml,
- fastai>=2.4,
- fastcore>=1.3.27

You can either install these packages manually or specify `["fastai"]` when installing `huggingface_hub` and your environment will be ready:

```
pip install huggingface_hub["fastai"]
```

Note: As of May 5, 2022, there has not been a release that includes the fastai+hub features, so you can install from `main`:

```
!pip install git+https://github.com/huggingface/huggingface_hub#egg=huggingface-hub["fastai"]
```

To share models in the Hub, you will need to have a user. You can create it on the [Hugging Face website](https://huggingface.co/).


## Sharing a Learner to the Hub


In [ ]:
#export
def push_to_hub_fastai(
    learner,
    repo_id: str,
    commit_message: Optional[str] = "Add model",
    private: Optional[bool] = None,
    token: Optional[str] = None,
    config: Optional[dict] = None,
    **kwargs,
):
    """
    Upload learner checkpoint files to the Hub while synchronizing a local clone of the repo in `repo_id`.
    
    Returns:
        The url of the commit of your model in the given repository.

    Raises the following error:
        - [`ValueError`](https://docs.python.org/3/library/exceptions.html#ValueError)
          if the user is not log on to the Hugging Face Hub.
    """

    _check_fastai_fastcore_versions()

    api_endpoint: str = kwargs.get("api_endpoint", None)
    git_user: str = kwargs.get("git_user", None)
    git_email: str = kwargs.get("git_email", None)

    if token is None:
        token = HfFolder.get_token()

    if token is None:
        raise ValueError(
            "You must login to the Hugging Face Hub. There are two options: "
            "(1) Type `huggingface-cli login` in your terminal and enter your token. "
            "(2) Enter your token in the `token` argument. "
            "Your token is available in the Settings of your Hugging Face account. "
        )

    # Create repo using `HfApi()`.
    repo_url = HfApi(endpoint=api_endpoint).create_repo(
        repo_id,
        token=token,
        private=private,
        repo_type=None,
        exist_ok=True,
    )

    # If repository exists in the Hugging Face Hub then clone it locally in `repo_id`.
    repo = Repository(
        repo_id,
        clone_from=repo_url,
        use_auth_token=token,
        git_user=git_user,
        git_email=git_email,
    )
    repo.git_pull(rebase=True)

    _save_pretrained_fastai(learner, repo_id, config=config)

    return repo.push_to_hub(commit_message=commit_message)

First, login to the Hugging Face Hub. There are two options:
1. Type `huggingface-cli login` in your terminal and enter your token. 
2. Enter your token in the `token` argument of the `push_to_hub_fastai` function.

```
from huggingface_hub import notebook_login
notebook_login()
```

Your token is available in your Hugging Face Account Settings.

Use `push_to_hub_fastai` with the learner you want to upload and the repository id for your learner in the Hub in the format of "namespace/repo_name". The namespace can be your individual account or an organization to which you have write access (for example, 'fastai/stanza-de').

```
from huggingface_hub import push_to_hub_fastai
push_to_hub_fastai(learner=learn, repo_id="espejelomar/cool_learner")
```

It is as simple as that! Now you can find your Learner right in the Hub with the id `espejelomar/cool_learner`.

When uploading a fastai Learner (or any other model) to the Hub, don't forget to edit its model card (image below). Here is [Hugging Face documentation](https://huggingface.co/docs/hub/model-repos#what-are-model-cards-and-why-are-they-useful) with everything you need.

<img src="images/hf_model_card.png" alt="hf_model_card" width="800" />



## Loading a Learner from the Hugging Face Hub


In [ ]:
#export
def from_pretrained_fastai(
    repo_id: str,
    revision: Optional[str] = None,
):
    """
    Load pretrained fastai model from the Hub or from a local directory. `repo_id` is the location where the pickled fastai.Learner is. It can be either of the two:
                - Hosted on the Hugging Face Hub. E.g.: 'espejelomar/fatai-pet-breeds-classification' or 'distilgpt2'.
                  You can add a `revision` by appending `@` at the end of `repo_id`. E.g.: `dbmdz/bert-base-german-cased@main`.
                  Revision is the specific model version to use. Since we use a git-based system for storing models and other
                  artifacts on the Hugging Face Hub, it can be a branch name, a tag name, or a commit id.
                - Hosted locally. `repo_id` would be a directory containing the pickle and a pyproject.toml
                  indicating the fastai and fastcore versions used to build the `fastai.Learner`. E.g.: `./my_model_directory/`.

    Returns:
        The `fastai.Learner` model in the `repo_id` repo.
    """
    _check_fastai_fastcore_versions()

    # `snapshot_download` returns the folder where the model was stored.
    if not os.path.isdir(repo_id):
        storage_folder = snapshot_download(
            repo_id=repo_id,
            revision=revision,
            library_name="fastai",
            library_version=get_fastai_version(),
        )
    else:
        storage_folder = repo_id

    _check_fastai_fastcore_pyproject_versions(storage_folder)

    from fastai.learner import load_learner

    return load_learner(os.path.join(storage_folder, "model.pkl"))

Loading a model from the Hub is even simpler. To load a model in the Hub with the id `ITESM/fastai_model`:

```
from huggingface_hub import from_pretrained_fastai
learner = from_pretrained_fastai("ITESM/fastai_model")
```

## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()